In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

#!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup

#!mamba install -c conda-forge html5lib --yes
# import html5lib
from geopy.geocoders import Nominatim

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##### Scraping wiki url and create data frame

In [2]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

page = requests.get(wiki_url)
soup = BeautifulSoup(page.content, 'html.parser')

headings = ['Postal Code','Borough','Neighbourhood']


wiki_table = soup.find('table', class_= "wikitable sortable")

data = {'Postal Code':[], 'Borough':[],'Neighbourhood':[]}

rows = wiki_table.find_all('tr')

#print('number of rows: {}'.format(len(rows)))

row_marker = 0
for row in rows:
    column_marker = 0
    columns = row.find_all('td')
    for key, column in zip(data,columns):
        check = column.get_text()
        data[key].append(check[:-1])

df = pd.DataFrame(data)
df.head(11)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


##### Dropping rows with Borough 'Not assigned'

In [3]:
indexNames = df[df['Borough']== 'Not assigned'].index
indexNames
df_v2 = df.drop(indexNames).reset_index(drop=True)
df_v2.head(5)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##### For rows with **Not assigned** Neighbourhood, adopt Borough value for Neighbourhood

In [4]:
indexNames2 = df_v2[df_v2['Neighbourhood']=='Not assigned'].index
df_v3 = df_v2

for ind in df_v2.index:
    if ind in indexNames2:
        df_v3.iloc[ind]['Neighbourhood']=df_v3.iloc[ind]['Borough']

df_v3.head(11)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


##### Checking shape of final dataframe

In [5]:
df_v3.shape

(103, 3)

In [6]:
# #!mamba install -c conda-forge geocoder --yes
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# for ind in df_v3.index:
#     postal_code = df_v3.iloc[ind]['Postal Code']
    
#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#         lat_lng_coords = g.latlng
    
#     latitude = lat_lng_coords[0]
#     longitude = lat_lng_coords[1]
#     df_v3[ind]['Latitude'] = latitude
#     df_v3[ind]['Longitude'] = longitude

# df_v3.head(10)

##### Reading Geospatial_Coordinates.csv into a new dataframe

In [7]:
df_coord = pd.read_csv('http://cocl.us/Geospatial_data')

##### Creating a new dataframe to include columns for geospatial coordinates

In [8]:
df_v4 = pd.merge(df_v3,df_coord, on = 'Postal Code', how='inner')
df_v4.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
df_v5 = df_v4[df_v4['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_v5.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [10]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_v5['Latitude'], df_v5['Longitude'], df_v5['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'O1UZY5PJ2QNASR1O5I3EL5W10RHZ2XBDHYZM1NYW4WBEYDSH' # your Foursquare ID
CLIENT_SECRET = 'K2PJ4JKQSFJA2CJPGJ4REM4NQA33VWHX0GL1QL5XZ52TWQUU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O1UZY5PJ2QNASR1O5I3EL5W10RHZ2XBDHYZM1NYW4WBEYDSH
CLIENT_SECRET:K2PJ4JKQSFJA2CJPGJ4REM4NQA33VWHX0GL1QL5XZ52TWQUU


#### Let's explore the first neighbourhood in dataframe

In [13]:
df_v5.loc[0,'Neighbourhood']

'Regent Park, Harbourfront'

#### Get neighbourhood's latitude and longitude values

In [14]:
neighbourhood_latitude = df_v5.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df_v5.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df_v5.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


#### Now, let's get the top 10 venues that are in Regent Park within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [16]:
# type your answer here

LIMIT = 10 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=O1UZY5PJ2QNASR1O5I3EL5W10RHZ2XBDHYZM1NYW4WBEYDSH&client_secret=K2PJ4JKQSFJA2CJPGJ4REM4NQA33VWHX0GL1QL5XZ52TWQUU&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=10'

Send the GET request and examine the result

In [ ]:
results = requests.get(url).json()
results

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [19]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/thomaschuajk/anaconda3/envs/projectenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


And how many venues were returned by Foursquare?

In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


## Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighbourhood and create a new dataframe called *toronto_venues*.

In [22]:
toronto_venues = getNearbyVenues(names=df_v5['Neighbourhood'],
                                   latitudes=df_v5['Latitude'],
                                   longitudes=df_v5['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


#### Let's check the size of the resulting dataframe

In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(348, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let's check how many venues were returned for each neighborhood

In [24]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,10,10,10,10,10,10
"Brockton, Parkdale Village, Exhibition Place",10,10,10,10,10,10
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",10,10,10,10,10,10
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",10,10,10,10,10,10
Central Bay Street,10,10,10,10,10,10
Christie,10,10,10,10,10,10
Church and Wellesley,10,10,10,10,10,10
"Commerce Court, Victoria Hotel",10,10,10,10,10,10
Davisville,10,10,10,10,10,10


#### Let's find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 127 uniques categories.


## Analyze Each Neighborhood

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Arts & Crafts Store,Asian Restaurant,...,Swim School,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [27]:
toronto_onehot.shape

(348, 128)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Arts & Crafts Store,Asian Restaurant,...,Swim School,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.00,0.1,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.1,0.1,0.1,0.1,0.1,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
5,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
6,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.1,0.00,0.0,0.0,0.0,0.0
7,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
8,Davisville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
9,Davisville North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


#### Let's confirm the new size

In [29]:
toronto_grouped.shape

(39, 128)

#### Let's print each neighborhood along with the top 5 most common venues

In [30]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0          Museum   0.1
1  Farmers Market   0.1
2    Concert Hall   0.1
3        Beer Bar   0.1
4      Restaurant   0.1


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0         Coffee Shop   0.2
1      Breakfast Spot   0.1
2  Italian Restaurant   0.1
3                 Gym   0.1
4        Climbing Gym   0.1


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
           venue  freq
0        Brewery   0.1
1    Pizza Place   0.1
2     Comic Shop   0.1
3     Skate Park   0.1
4  Burrito Place   0.1


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0             Airport   0.1
1  Airport Food Court   0.1
2         Coffee Shop   0.1
3            Boutique   0.1
4                 Bar   0.1


----Central Bay Street----
                       venue  freq
0                Coffee Shop   

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Park,Liquor Store,Vegetarian / Vegan Restaurant,Cocktail Bar,French Restaurant,Beer Bar,Museum,Concert Hall,Restaurant,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Breakfast Spot,Bakery,Italian Restaurant,Pet Store,Gym,Climbing Gym,Bar,Café,Dog Run
2,"Business reply mail Processing Centre, South C...",Garden Center,Auto Workshop,Brewery,Restaurant,Farmers Market,Fast Food Restaurant,Burrito Place,Skate Park,Pizza Place,Comic Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport,Harbor / Marina,Coffee Shop,Plane,Bar,Boutique,Airport Food Court,Airport Terminal,Airport Lounge,Airport Gate
4,Central Bay Street,Coffee Shop,Modern European Restaurant,Bubble Tea Shop,Sushi Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Gastropub,Cuban Restaurant,Creperie,Cosmetics Shop


## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [33]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 2, 1, 0, 1, 2, 1, 1, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [34]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_v5

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Gym / Fitness Center,Restaurant,Spa,Pub,Bakery,Historic Site,Breakfast Spot,Distribution Center,Park
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Yoga Studio,Sushi Restaurant,Coffee Shop,Persian Restaurant,Creperie,Portuguese Restaurant,Distribution Center,Italian Restaurant,Beer Bar,Park
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Café,Burrito Place,Clothing Store,Music Venue,Comic Shop,Theater,Pizza Place,Burger Joint,Plaza,Yoga Studio
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Gastropub,Coffee Shop,Cosmetics Shop,Creperie,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Italian Restaurant,Gym,Food Truck
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Neighborhood,Trail,Pub,Yoga Studio,Diner,Dessert Shop,Department Store,Dance Studio,Cuban Restaurant


Finally, let's visualize the resulting clusters

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

### Cluster 1

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,0,Coffee Shop,Modern European Restaurant,Bubble Tea Shop,Sushi Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Gastropub,Cuban Restaurant,Creperie,Cosmetics Shop
31,Central Toronto,0,Coffee Shop,Sushi Restaurant,Liquor Store,Fried Chicken Joint,Restaurant,American Restaurant,Pub,Sports Bar,Supermarket,Department Store


### Cluster 2

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Gym / Fitness Center,Restaurant,Spa,Pub,Bakery,Historic Site,Breakfast Spot,Distribution Center,Park
1,Downtown Toronto,1,Yoga Studio,Sushi Restaurant,Coffee Shop,Persian Restaurant,Creperie,Portuguese Restaurant,Distribution Center,Italian Restaurant,Beer Bar,Park
2,Downtown Toronto,1,Café,Burrito Place,Clothing Store,Music Venue,Comic Shop,Theater,Pizza Place,Burger Joint,Plaza,Yoga Studio
3,Downtown Toronto,1,Gastropub,Coffee Shop,Cosmetics Shop,Creperie,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Italian Restaurant,Gym,Food Truck
7,Downtown Toronto,1,Café,Grocery Store,Candy Store,Italian Restaurant,Diner,Restaurant,Coffee Shop,Cuban Restaurant,Creperie,Dog Run
9,West Toronto,1,Bakery,Bank,Music Venue,Grocery Store,Gym / Fitness Center,Café,Middle Eastern Restaurant,Brewery,Bar,Yoga Studio
12,East Toronto,1,Italian Restaurant,Greek Restaurant,Ice Cream Shop,Yoga Studio,Brewery,Cosmetics Shop,Juice Bar,Diner,Dog Run,Distribution Center
13,Downtown Toronto,1,Coffee Shop,Gastropub,Gym / Fitness Center,Restaurant,Hotel,Pub,Bakery,Café,Gym,Dance Studio
14,West Toronto,1,Coffee Shop,Breakfast Spot,Bakery,Italian Restaurant,Pet Store,Gym,Climbing Gym,Bar,Café,Dog Run
16,Downtown Toronto,1,Café,Gastropub,Bakery,Coffee Shop,Museum,Restaurant,Gym,Gym / Fitness Center,Pub,Dessert Shop


### Cluster 3

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,2,Health Food Store,Neighborhood,Trail,Pub,Yoga Studio,Diner,Dessert Shop,Department Store,Dance Studio,Cuban Restaurant
5,Downtown Toronto,2,Park,Liquor Store,Vegetarian / Vegan Restaurant,Cocktail Bar,French Restaurant,Beer Bar,Museum,Concert Hall,Restaurant,Farmers Market
8,Downtown Toronto,2,Hotel,Café,Restaurant,Speakeasy,Plaza,Steakhouse,Gym / Fitness Center,Asian Restaurant,Concert Hall,Vegetarian / Vegan Restaurant
10,Downtown Toronto,2,Park,Plaza,Neighborhood,Skating Rink,Hotel,Salad Place,Sporting Goods Shop,Dessert Shop,Lake,Performing Arts Venue
11,West Toronto,2,New American Restaurant,Pizza Place,Korean Restaurant,Beer Store,Ice Cream Shop,Brewery,Asian Restaurant,Cuban Restaurant,Greek Restaurant,Wine Bar
15,East Toronto,2,Park,Brewery,Italian Restaurant,Ice Cream Shop,Pub,Liquor Store,Fast Food Restaurant,Fish & Chips Shop,Sushi Restaurant,Gym
20,Central Toronto,2,Park,Food & Drink Shop,Department Store,Hotel,Breakfast Spot,Gym / Fitness Center,Sandwich Place,Pizza Place,Concert Hall,Comic Shop
21,Central Toronto,2,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Dance Studio,Distribution Center,Diner,Dessert Shop,Department Store,Yoga Studio
29,Central Toronto,2,Tennis Court,Restaurant,Park,College Gym,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dog Run,Cuban Restaurant
33,Downtown Toronto,2,Park,Trail,Playground,Cuban Restaurant,Distribution Center,Diner,Dessert Shop,Department Store,Dance Studio,Yoga Studio
